In [1]:
import tensorflow as tf
import numpy as np
import pickle

### Encoding

In [2]:
# HTM SDR Scalar Encoder
# Input: Scalar
# Parameters: n - number of units, w - bits used to represent signal (width), b - buckets (i.e. resolution), 
#             min - minimum value of input (inclusive), max - maximum input value (inclusive)
class scalar_sdr:
    
    def __init__(self, b, w, min_, max_, shape=(1,1), neg=True):
        if type(b) != int or type(w) != int or type(min_) != float or type(max_) != float:
            raise TypeError("b - buckets must be int, w - width must be int, min_ must be float and max_ must be float")
        self.b = b # must be int
        self.w = w # must be int
        self.min = min_ # must be float
        self.max = max_ # must be float
        self.n = b+w-1 # number of units for encoding
        self.ndarray_shape = shape
        self.nodes = self.n*reduce(lambda x, y: x*y, self.ndarray_shape)
        self.neg = neg
        
    def encode(self,input_):
        if input_ > self.max or input_ < self.min:
            raise ValueError("Input outside encoder range!")
        if type(input_) != float:
            raise TypeError("Input must be float!")
        if self.neg:
            output = np.zeros(self.n)-1
        else:
            output = np.zeros(self.n)
        index = int((input_-self.min)/(self.max-self.min)*self.b)
        output[index:index+self.w] = 1
        return output
    
    def encode_ndarray(self,input_):
        if input_.shape != self.ndarray_shape:
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        output = []
        for i in np.nditer(input_, order='K'):
            output.append(self.encode(float(i)))
        return np.array(output)
    '''
    def decode(self,input_):
        if len(input_) != self.n: # or len(np.nonzero(input_+1)[0]) != self.w: <-- Can't have since the network is not guaranteed to produce this by any means!!!
            raise TypeError("Input does not correspond to encoder encoded data!")
        # output = np.nonzero(input_+1)[0][0]/float(self.b)*(self.max-self.min)+self.min <-- This doesn't work really since bits can randomly fire, taking the average is a more reasonable decoding
        median = np.median(np.nonzero(input_+1)[0])            
        try:
            output = int(median-float(self.w)/2.0)/float(self.b)*(self.max-self.min)+self.min # i.e. figure out center (median more outlier resistant than mean) and subtract width/2
        except ValueError:
            output = None
        return output
    '''
    def decode(self,input_):
        if len(input_) != self.n: 
            raise TypeError("Input length does not match encoder length!")
        if len(np.nonzero(input_+1)[0]) == 0:
            return np.nan
        max_ = 0
        output = 0.0
        for i in range(self.b):
            x = np.zeros(self.n)-1
            x[i:i+self.w] = 1
            if x.shape != input_.shape:
                input_ = input_.reshape(x.shape)
            score = np.inner(x,input_) # this was broken for some input formats 
            if score > max_:
                max_ = score
                output = float(i)/float(self.b)*(self.max-self.min)+self.min
        return output
            
    def decode_ndarray(self,input_):
        if input_.shape != (reduce(lambda x, y: x*y, self.ndarray_shape)*self.n,): 
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        input_ = input_.reshape(self.ndarray_shape+(self.n,))
        output = []
        for i in np.ndindex(self.ndarray_shape):
            output.append(self.decode(input_[i]))
        output = np.array(output).reshape(self.ndarray_shape)
        return output
    
    def set_ndarray_shape(self,shape):
        if type(shape) != tuple:
            raise TypeError("Must provide tuple of array dimensions!")
        self.ndarray_shape = shape

### Behavior data

In [3]:
fb = pickle.load(open("/home/markus/dep/dep_data/bases/fb.pickle","rb"))
fs = pickle.load(open("/home/markus/dep/dep_data/bases/fs.pickle","rb"))
sd = pickle.load(open("/home/markus/dep/dep_data/bases/sd.pickle", "rb"))

bases = {"fb": fb, "fs": fs, "sd": sd}

### Hard coded neurons

In [54]:
active_motors = [1,3,4,5,10,12]
ts = {'fb': 124, 'fs': 126, 'sd': 117}

In [56]:
pos_encoder = scalar_sdr(30,1,-100000.0,100000.0,neg=False)
vel_encoder = scalar_sdr(10,1,-70.0,70.0,neg=False)
weights = []
weights_vel = []
for i in active_motors:
    w = np.zeros(pos_encoder.nodes)
    w_vel = np.array([np.zeros(vel_encoder.nodes)-1 for j in range(pos_encoder.nodes)])
    for id_ in bases:
        val = bases[id_][0][ts[id_]][i]
        index_pos = int((val-pos_encoder.min)/(pos_encoder.max-pos_encoder.min)*pos_encoder.b)
        w[index_pos] = 1

        val = bases[id_][1][ts[id_]][i]
        index_vel = int((val-vel_encoder.min)/(vel_encoder.max-vel_encoder.min)*vel_encoder.b)
        w_vel[index_pos][index_vel] = 0
        
    weights.append(w)
    weights_vel.append(w_vel)
weights = np.array(weights)
weights_vel = np.array(weights_vel)
weights = weights.reshape(6,30,1)
weights_vel = weights_vel.reshape(6,30,10)

In [96]:
pos_encoder = scalar_sdr(30,1,-100000.0,100000.0,shape=(6,1),neg=False)
vel_encoder = scalar_sdr(10,1,-70.0,70.0,shape=(6,1),neg=False)

In [97]:
def testTrigger(id_,index):
    #index = ts[id_]
    pos_ = pos_encoder.encode_ndarray(bases[id_][0][index][active_motors].reshape(6,1)).reshape(6,30,1)
    vel_ = vel_encoder.encode_ndarray(bases[id_][1][index][active_motors].reshape(6,1)).reshape(6,10,1)
    
    with tf.name_scope("input"):
        pos_nodes = tf.placeholder(tf.float32, [6,30,1], name="encoded_motor_positions")
        vel_nodes = tf.placeholder(tf.float32, [6,10,1], name="encoded_motor_velocities")
        trigger_bias = tf.constant(1., shape=[6,1], dtype=tf.float32)

    with tf.name_scope("weights"):
        w_pos = tf.Variable(np.zeros((6,30,1)), name="position_weights", dtype=tf.float32)
        w_vel = tf.Variable(np.zeros((6,30,10)), name="velocity_weights", dtype=tf.float32)
        w_trig = tf.constant(-(6.-1e-6)/6., shape=[6,1], name="trigger_weights", dtype=tf.float32)

    with tf.name_scope("trigger"):
        pos_sum = tf.multiply(pos_nodes,w_pos)
        vel_sum = tf.matmul(w_vel,vel_nodes)
        motor_sum = pos_sum + vel_sum
        motor_activation = tf.nn.relu(tf.sign(motor_sum))
        trigger_sum_0 = tf.reduce_sum(motor_activation, axis = 1)
        trigger_sum_1 = tf.reduce_sum(trigger_sum_0 + tf.multiply(trigger_bias,w_trig))
        trigger_activation = tf.nn.relu(tf.sign(trigger_sum_1))

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    return sess.run(trigger_activation,{pos_nodes: pos_, w_pos: weights, vel_nodes: vel_, w_vel: weights_vel})
    # Create a summary writer, add the 'graph' to the event file.
    # writer = tf.summary.FileWriter("tf_logs", sess.graph)

In [99]:
trigger = {}
for base in bases:
    trigger[base] = np.zeros(bases[base][0].shape[0])
    for i in range(bases[base][0].shape[0]):
        print i
        trigger[base][i] = testTrigger(base,i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


In [101]:
trigger

{'fb': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]),
 'fs': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,